## Programación dinámica discreta deterministica

Esta es una implementación simple del modelo de programación dinámica discreta determinística. 
### Código

In [30]:
pddd0 <- 
function(numEtapas,  # numero de etapas 
         numDiscr,   # numero de discretizaciones
         volMax,     # volumen maximo del embalse
         volMin = 0, # volumen minimo del embalse
         volInit,    # volumen inicial
         volFinish,  # volumen final
         P = 0,      # penalización
         GHmax,      # generacion hidraulica maxima (energia)
         GTmax,      # generación termica maxima (energia)
         CC,         # costo de combustibles
         CR,         # costo de racionamiento
         aportes,    # aportes hidrologicos (energia)
         demanda     # en energia
        )
    {
    # 
    # trajectory[i, j] = k
    #
    #     indica que la decision optima es pasar de la
    #     j-esima discretizacion en la etapa i-1
    #     a la k-esima discretizacion a la etapa i
    #
    trajectory  = matrix(0, numEtapas, numDiscr+1)
    
    #
    # FCT[i, j] = $
    #     
    #     es el costo total ($) en que se ha incurrido en la etapa i
    #     y la discretizacion j. La matriz es inicializada en NA
    #
    FCT = matrix(NA, numEtapas, numDiscr+1)
    
    # valores del volumen para cada discretizacion
    volumenes = volMin + (volMax - volMin) / numDiscr * (0:numDiscr)
    
    #
    # calcula los volumenes inicial y final en terminos
    # de las discretizaciones
    #
    delta       = (volMax - volMin) / numDiscr
    discrInit   = 1 + round((volInit   - volMin) / delta, 0)
    discrFinish = 1 + round((volFinish - volMin) / delta, 0)
    
    #
    # optimizacion
    #
    
    # etapa es la etapa actual
    for (etapa in (numEtapas:1))
        {
        # i es la discretizacion del volumen de la etapa anterior
        for (i in 1:(numDiscr+1))
            {
            # j es la discretizacion del volumen de la etapa actual
            for (j in 1:(numDiscr+1))
                {

                if (etapa == numEtapas & j != discrFinish)
                        #
                        # en la ultima etapa solo se considera cuando 
                        # se llega al volumen final deseado
                        #
                        next
                
                if (etapa == 1 & i != discrInit)
                        # 
                        # en la primera etapa solo se consideran cuando
                        # los casos que tienen el volumen inicial igual
                        # al deseado
                        # 
                        next

    
                Vprevio = volumenes[i]
                Vactual = volumenes[j]
                A = aportes[etapa]

                if (Vprevio + A - Vactual < 0)
                    #
                    # inviable
                    #
                    next
    

    
                GH = min(GHmax, demanda[etapa], Vprevio + A - Vactual)
                S = Vprevio + A - GH - Vactual # >= 0
    
                GT   = min(GTmax, demanda[etapa] - GH)
                R    = demanda[etapa] - GH - GT
                FCIo = GT * CC + R * CR	
                    
                if (etapa == numEtapas)
                    {
                    FCTo = FCIo
                    }
                else
                    {
                    FCTo = FCIo + FCT[etapa+1, j]
                    }
                    
                if (is.na(FCT[etapa, i]) | FCT[etapa, i] > FCTo)
                    {
                    FCT[etapa, i] = FCTo
                    trajectory[etapa, i] = j
                    }           
                }
            }
        }

 
    sol_volumenes = c()
    sol_GH = c()
    sol_GT = c()
    sol_R  = c()
    sol_S  = c()

    i = discrInit
    for (etapa in 1:numEtapas)
        {
        j = trajectory[etapa, i]
        Vprevio = volumenes[i]
        Vactual = volumenes[j]

        GH = min(GHmax, demanda[etapa], volumenes[i] + aportes[etapa] - volumenes[j])
        S  = volumenes[i] + aportes[etapa] - GH - volumenes[j]
        GT = min(GTmax, demanda[etapa] - GH)
        R  = demanda[etapa] - GH - GT
        
        sol_volumenes = c(sol_volumenes, volumenes[j])
        sol_GH        = c(sol_GH, GH)
        sol_S         = c(sol_S, S)
        sol_GT        = c(sol_GT, GT)
        sol_R         = c(sol_R, R)
        
        i = j

        }
        
    return (list(volumenes = sol_volumenes,
                 GH = sol_GH,
                 GT = sol_GT,
                 Vertimientos = sol_S,
                 Racionamientos = sol_R))
    }



# Ejemplo de clase

A continuación se realiza la corrida para el ejemplo de clase. Note que los resultados son devueltos en una variable. Contraste los resultados

In [31]:
r = pddd0( numEtapas = 4, 
           numDiscr  = 4,  
           volMax    = 100,
           volMin    = 0, 
           volInit   = 75,
           volFinish = 0,
           P         = 0, 
           GHmax     = 50,
           GTmax     = 45,
           CC        = 15,
           CR        = 1000,
           aportes   = c(21, 15, 12, 18), 
           demanda   = c(50, 50, 50, 50))

r

$volumenes
[1] 50 25  0  0

$GH
[1] 46 40 37 18

$GT
[1]  4 10 13 32

$Vertimientos
[1] 0 0 0 0

$Racionamientos
[1] 0 0 0 0